In [ ]:
"""
# 파이썬 가상환경 생성
python -m venv .venv
  
(.venv) = 설정하고자 가상환경 이름

# 주피터 커널 등록
python -m ipykernel install --user --name=venv --display-name "Python (.venv)
(Python) = 설정하고자 하는 가상환경 이름


PowerShell에서 가상환경 활성화
.\.venv\Scripts\Activate.PS1

가상환경 리스트 확인 명령어
jupyter kernelspec list

"""

### PyMuPDF + Camelot(Lattice)

In [1]:
import camelot
import fitz
import pandas as pd
from langchain.schema import Document
from tabulate import tabulate
import os
def extract_text_with_fitz(pdf_path):
    
    doc = fitz.open(pdf_path)
    texts = []
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()
        texts.append(text.strip() if text else "")
        
    return texts

def extract_tables_lattice_per_page(pdf_path):
    # 페이지별 표를 저장할 딕셔너리 {페이지번호(1-based): [DataFrame, ...], ...}
    tables_per_page = {}

    # Camelot으로 모든 표를 추출 (페이지별로 추출을 못하니 각 표의 페이지 정보 이용)
    tables = camelot.read_pdf(pdf_path, pages="1-end", flavor="lattice")
    
    for table in tables:
        page_num = table.page  # Camelot이 알려주는 페이지 번호 (1-based)
        df = table.df.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
        if page_num not in tables_per_page:
            tables_per_page[page_num] = []
        tables_per_page[page_num].append(df)
        

    return tables_per_page

def convert_tables_to_markdown(tables):
    markdowns = []
    for i, df in enumerate(tables):
        try:
            markdown_table = tabulate(df.values.tolist(), headers=df.iloc[0].tolist(), tablefmt="github")
            markdowns.append(f"▼ 표 {i + 1}:\n{markdown_table}")
        except Exception as e:
            markdowns.append(f"▼ 표 {i + 1}: 변환 실패 ({e})")
    return "\n\n".join(markdowns)
            

# 경로
pdf_path = "./data/2025년 AI반도체 조기 상용화 및 AX실증 지원 사업 통합 공고문.pdf"

# 파일명, 확장자 분리
file_with_ext = os.path.basename(pdf_path)
file_name, file_ext = os.path.splitext(file_with_ext)

# 텍스트 추출
texts = extract_text_with_fitz(pdf_path)

# 페이지별 표 추출
tables_per_page= extract_tables_lattice_per_page(pdf_path)


# 페이지별 document 리스트 생성
documents = []
total_pages = len(texts)

for i, text in enumerate(texts):
    page_num = i + 1
    tables = tables_per_page.get(page_num, [])
    
    tables_Str = convert_tables_to_markdown(tables)
    combined_text = text
    
    if tables_Str:
        combined_text += "\n\n---\n\n📊 페이지 내 표 정보:\n" + tables_Str
    
    doc = Document(
        page_content=combined_text, 
        metadata={"page": page_num,
                  "title": file_name
                  }
    )
    documents.append(doc)

# 예시 출력
with open("test/pdf.txt", "w", encoding="utf-8") as f:
    f.write(documents[3].page_content)

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



PdfReadError("Invalid Elementary Object starting with b')' @179086: b'ttp://smart.nipa.kr))\\n/S /URI >>\\n/H /I\\n/F 28\\n>>\\nendobj\\n311 0 obj\\n<< /Type /Annot'")
PdfReadError("Invalid Elementary Object starting with b')' @179278: b'(http://www.nipa.kr))\\n/S /URI >>\\n/H /I\\n/F 28\\n>>\\nendobj\\n309 0 obj\\n[\\n308 0 R\\n310 0'")
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23468\2052424735.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = table.df.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)


In [2]:
#  단계 2: 문서 분할
from langchain.text_splitter import NLTKTextSplitter

text_splitter = NLTKTextSplitter()

split_documents = text_splitter.split_documents(documents)

split_documents[0].metadata

{'page': 1, 'title': '2025년 AI반도체 조기 상용화 및 AX실증 지원 사업 통합 공고문'}

In [3]:
# 단계 3: 임베딩 생성

# sentence-transformers 라이브러리를 사용하면  (위에서 정의함)
# HuggingFace 모델에서 사용된 사전 훈련된 임베딩 모델을 다운로드 받아서 적용

from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli', # 한국어 자연어 추론 최족화된  ko-sroberta 모델
    model_kwargs={'device':'cpu'}, # CPU에서 실행되도록 설정
    encode_kwargs={'normalize_embeddings':True}, # 임베딩을 정규화, 벡터가 같은 범위의 값을 갖도록 함. (유사도 계산시 일관성 높임)
)

embeddings_model

c:\Users\USER\Desktop\KSVA\Project\python\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-nli', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [3]:
# 단계 4 : DB 생성 ChromaDB 및 저장
from langchain_community.vectorstores import Chroma

# Chroma 벡터 스토어에 문서와 메타데이터 저장
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings_model,
    persist_directory="./chroma_db"  # 벡터 스토어를 디스크에 저장
)

vectorstore

NameError: name 'split_documents' is not defined

In [4]:
# 단계 5: 검색기(Retriver) 생성
retriever = vectorstore.as_retriever()
retriever

NameError: name 'vectorstore' is not defined

In [2]:
# MMR Retriever 적용
retriever = vectorstore.as_retriever(
    search_type="mmr",              # MMR 기반 유사도 검색
    search_kwargs={
        "k": 10,                    # 전체 후보 문서 수
        "fetch_k": 30,              # 더 많은 후보 중 다양성 고려해 k개 선택
        "lambda_mult": 0.7          # 유사도 vs 다양성 균형 (1.0: 유사도만, 0.0: 다양성만)
    }
)

NameError: name 'vectorstore' is not defined

In [6]:
# 단계 6: 프롬프트 생성(Create Prompt)
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Please answer the question in Korean

    #Question:
    {question}

    #Context:
    {context}

    #Answer:
    """
)

print(type(prompt))

<class 'langchain_core.prompts.prompt.PromptTemplate'>


In [7]:
# LLM 모델 로드 - ChatGPT

from langchain.chat_models import ChatOpenAI
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# 객체 생성
llm = ChatOpenAI(
    openai_api_key=api_key,
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-3.5-turbo",  # 모델명
)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23468\3632315121.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [1]:
# 단계 8 체인(Chain) 생성
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

NameError: name 'retriever' is not defined

In [10]:
# 단계 9 답변 생성
question = 'AI 지원사업 지원자격 알려줘?'

response = chain.invoke(question)
print(response)

AI 지원사업의 지원자격은 AI컴퓨팅 실증 인프라 고도화 사업의 과업수행에 문제가 없는 기업 및 기관이 해당됩니다. 또한 국산 AI반도체 기반 AI컴퓨팅 인프라 구축, AI응용서비스 실증 수행, 클라우드 서비스 인프라 운영 및 제공, AI-IaaS 서비스 구축 및 운영 등의 역할을 수행할 수 있는 자격이 필요합니다.


# ./data/ -> pdf 모든거 파일 임베딩하기

In [ ]:
#### 자동화
import os
import fitz
import camelot
from langchain.schema import Document
from tabulate import tabulate

# PymuPDF로 텍스트 분할
def extract_text_with_fitz(pdf_path):
    doc = fitz.open(pdf_path) # PDF 문서 열기
    texts = []
    
    # PDF 문서 내 모든 페이지 순회
    for page_num in range(len(doc)):
        page = doc.load_page(page_num) # 페이지 단위로 로드 (0부터 시작)
        text = page.get_text() # 텍스트 추출
        texts.append(text.strip() if text else "") # 공백 제거 후 저장
    return texts


# camelot - lattice로 표 자르기
def extract_tables_lattice_per_page(pdf_path):
    
    tables_per_page = {} # {페이지번호: [DataFrame, ...], ...} 형태 저장
    
    tables = camelot.read_pdf(pdf_path, pages="1-end", flavor="lattice") 
    # 모든 페이지 대상 표 추출
    # (격자무늬 구조 표 추출에 적합)
    
    for table in tables:
        page_num = table.page # Camelot이 인식한 페이지 번호(1부터 시작함)
        
        df = table.df.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
        # isinstance(x, str): x가 문자열인지 검사
        # 표 데이터를 DataFrame 형태로 얻음
        # 셀 내 줄바꿈 문자를 공백으로 치환하여 한줄로 변환
        
        # 해당 페이지 번호가 딕셔너리에 없으면 새 리스트로 초기화
        # ex) 3페이지에 첫 표가나오면, 3이라는 키가 없으니 생성하고 추가
        if page_num not in tables_per_page:
            tables_per_page[page_num] = []
            
        tables_per_page[page_num].append(df)
        
    return tables_per_page


# 표 DataFrame 리스트를 마크다운 형식 표 문자열로 변환
def convert_tables_to_markdown(tables):
    
    markdowns = []
    
    for i, df in enumerate(tables):
        try:
                
            # 첫행일 헤더로 사용해 github 스타일 마크다운 표 생성
            markdown_table = tabulate(df.values.tolist(), headers=df.iloc[0].tolist(), tablefmt="github")
            # df.values.tolist(): DataFrame 데이터를 2차원 리스트(리스트 안에 리스트)로 변환
            # df.iloc[0].tolist(): DataFrame 첫 행을 리스트로 변환 -> 표 헤더로 사용
            # tabulate(..., headers=..., tablefmt="github"): 데이터를 마크다운 형식으로 예쁘게 변환
            
            markdowns.append(f"▼ 표 {i + 1}:\n{markdown_table}")
            
        except Exception as e:
            markdowns.append(f"▼ 표 {i + 1}: 변환 실패 ({e})")
            
    return "\n\n".join(markdowns)


#모든 파일  자동화
def load_all_pdfs_in_data_dir(data_dir="./data"):
    
    documents = []

    # ./data 폴더 내 모든 pdf 파일 리스트
    pdf_files = [f for f in os.listdir(data_dir) if f.lower().endswith(".pdf")]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(data_dir, pdf_file)
        file_name, _ = os.path.splitext(pdf_file)

        texts = extract_text_with_fitz(pdf_path)
        tables_per_page = extract_tables_lattice_per_page(pdf_path)

        for i, text in enumerate(texts):
            page_num = i + 1
            tables = tables_per_page.get(page_num, [])
            tables_str = convert_tables_to_markdown(tables)
            combined_text = text
            if tables_str:
                combined_text += "\n\n---\n\n📊 페이지 내 표 정보:\n" + tables_str

            doc = Document(
                page_content=combined_text,
                metadata={
                    "source_file": file_name,
                    "page": page_num
                }
            )
            documents.append(doc)

    return documents


if __name__ == "__main__":
    all_documents = load_all_pdfs_in_data_dir("./data")

    # 임베딩 및 벡터 DB 저장용으로 사용 가능
    print(f"총 문서 페이지 수: {len(all_documents)}")
    print("예시 문서 메타데이터:", all_documents[0].metadata)
    print("예시 문서 내용 일부:", all_documents[0].page_content[:300])
